# **Agricultural Exports Categories Analysis**
*by Sergio Postigo and Víctor Diví*

## **4. Data cleaning**

In this stage we will clean the data and specifically the columns that we will use in the model(s) in the next section. Of course, we don´t need to clean all the columns, since many of them are not relevant for labeling the rows. So, let's first determine the columns to be used and justify why

| COLUMN                             | USEFUL  | JUSTIFICATION                                                                                                                                                                                                    |
|------------------------------------|---------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Partida Aduanera                   | NO      | For each customs code there is one description in *Descripcion de la Partida Aduanera*. This last carries more information about the product. So, we won´t take this attribute and consider the next one.        |
| Descripcion de la Partida Aduanera | **YES** | This is a general description about the product, so this carries valuable information for the labeling                                                                                                           |
| Aduana                             | NO      | The port from which the product is being shipped. For now, we won´t consider it for our models                                                                                                                   |
| DUA                                | NO      | This is a random generated code associated with the shipping, it does not carry information that can be captured                                                                                                 |
| Fecha                              | **YES** | Associating the date of shipping to a category is insightful. As we saw, some products are exported in specific seasons of the year                                                                              |
| Año                                | NO      | Already included in the previous attribute                                                                                                                                                                       |
| Cod. Tributario                    | NO      | There is one tax code for each company. A company can be associated to specific groups of products, however the amount of different companies can be huge.                                                       |
| Exportador en Perú                 | NO      | Same idea as previous row                                                                                                                                                                                        |
| Importador Extranjero              | NO      | The amount of different importers abroad may be huge and new data my carry new names not learned by the model                                                                                                    |
| Kg Bruto                           | NO      | See next attribute                                                                                                                                                                                               |
| Kg Neto                            | **YES** | The weight of the shipments is insightful, but is highly variable among same products, so initially we won´t use this feature. However we will use it to calculate the price by kg, which is actually insightful |
| Toneladas Netas                    | NO      | See previous attribute                                                                                                                                                                                           |
| Qty 1                              | NO      | Same as before                                                                                                                                                                                                   |
| Und 1                              | NO      | Same as before                                                                                                                                                                                                   |
| Qty 2                              | NO      | Same as before                                                                                                                                                                                                   |
| Und 2                              | NO      | Same as before                                                                                                                                                                                                   |
| U$ FOB Tot                         | **YES** | The cost of the shipment will be use to calculate the cost by kg of the product                                                                                                                                  |
| Miles de USD Fob TOTAL             | NO      | It is just a repetition of the previous attribute                                                                                                                                                                |
| U$ FOB Und 1                       | NO      |                                                                                                                                                                                                                  |
| U$ FOB Und 2                       | NO      |                                                                                                                                                                                                                  |
| Pais de Destino                    | **YES** | The country were this products are being exported can be related to groups of products                                                                                                                           |
| Puerto de destino                  | NO      | The previous attribute indirectly captures this information already                                                                                                                                              |
| Último Puerto Embarque             | NO      |                                                                                                                                                                                                                  |
| Via                                | NO      |                                                                                                                                                                                                                  |
| Agente Portuario                   | NO      |                                                                                                                                                                                                                  |
| Agente de Aduana                   | NO      |                                                                                                                                                                                                                  |
| Descripcion Comercial              | **YES** | The comercial description also carries valuable information for the labeling                                                                                                                                     |
| Descripcion1                       | NO      | Already captured in *Descripcion Comercial*                                                                                                                                                                      |
| Descripcion2                       | NO      | Already captured in *Descripcion Comercial*                                                                                                                                                                      |
| Descripcion3                       | NO      | Already captured in *Descripcion Comercial*                                                                                                                                                                      |
| Descripcion4                       | NO      | Already captured in *Descripcion Comercial*                                                                                                                                                                      |
| Descripcion5                       | NO      | Already captured in *Descripcion Comercial*                                                                                                                                                                      |
| Naviera                            | NO      |                                                                                                                                                                                                                  |
| Agente Carga(Origen)               | NO      |                                                                                                                                                                                                                  |
| Agente Carga(Destino)              | NO      |                                                                                                                                                                                                                  |
| Canal                              | NO      |                                                                                                                                                                                                                  |
| Concatenar                         | NO      |                                                                                                                                                                                                                  |
| Categoría macro Aurum              | **YES** | **LABEL**                                                                                                                                                                                                        |
| Subcategoría inicial               | NO      | While we also need this category, it can be inferred given a prediction of the subcategory                                                                                                                       |
| Subcategoría Consolidada Aurum     | NO      |                                                                                                                                                                                                                  |
| Categoría Consolidada Aurum        | NO      |                                                                                                                                                                                                                  |

In [21]:
import pandas as pd

full_data = pd.read_csv("../data/raw_data/data.csv", encoding='latin-1', sep=';')

C:\Users\spost\AppData\Local\Temp\ipykernel_14000\713207146.py:3: DtypeWarning: Columns (6,14,16,22,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  full_data = pd.read_csv("../data/raw_data/data.csv", encoding='latin-1', sep=';')


In [22]:
data = full_data[
    ["Descripcion de la Partida Aduanera", "Fecha", "Kg Neto", "U$ FOB Tot", "Pais de Destino", "Descripcion Comercial",
     "Categoría macro Aurum"]].copy()
data.head()

,Descripcion de la Partida Aduanera,Fecha,Kg Neto,U$ FOB Tot,Pais de Destino,Descripcion Comercial,Categoría macro Aurum
0,"ACEITUNA CONSERVADAS PROVISIONALMENTE,TODAVIA ...",11/19/2019,8642,12291,Chile,ACEITUNAS ARAUCO PRESENTACION EN BIDONES RES...,Aceituna
1,"ACEITUNA CONSERVADAS PROVISIONALMENTE,TODAVIA ...",2/21/2019,10440,12362,Chile,ACEITUNAS ARAUCO PRESENTACION EN BIDONES VERDE...,Aceituna
2,"ACEITUNA CONSERVADAS PROVISIONALMENTE,TODAVIA ...",10/16/2018,480,1728,Australia,4 BIDONES ACEITUNA DE MESA ASCOLANA VERDE DESH...,Aceituna
3,ACEITUNAS,7/5/2018,1740,4002,Jordania,ACEITUNA ASCOLANA ENTERA 07-09 BID80L 58KG C/B...,Aceituna
4,"ACEITUNA CONSERVADAS PROVISIONALMENTE,TODAVIA ...",11/19/2019,2150,2826,Chile,ACEITUNA ASCOLANO PRESENTACION EN BIDONES RE...,Aceituna


From now on we will focus on each of the selected columns

#### **Descripcion de la Partida Aduanera (description of the customs code)**

In [23]:
data[["Descripcion de la Partida Aduanera"]]

,Descripcion de la Partida Aduanera
0,"ACEITUNA CONSERVADAS PROVISIONALMENTE,TODAVIA ..."
1,"ACEITUNA CONSERVADAS PROVISIONALMENTE,TODAVIA ..."
2,"ACEITUNA CONSERVADAS PROVISIONALMENTE,TODAVIA ..."
3,ACEITUNAS
4,"ACEITUNA CONSERVADAS PROVISIONALMENTE,TODAVIA ..."
...,...
631389,CALABAZAS (ZAPALLOS) Y CALABACINES (CUCURBITA ...
631390,CALABAZAS (ZAPALLOS) Y CALABACINES (CUCURBITA ...
631391,CALABAZAS (ZAPALLOS) Y CALABACINES (CUCURBITA ...
631392,CALABAZAS (ZAPALLOS) Y CALABACINES (CUCURBITA ...


Since in this column we are dealing with textual descriptions of the product, we will use Natural Language Processing techniques. A first important step that we will perform is to remove the so-called *stop words* from each cell, so that we get rid of the low-level information. For example, we see that the second row in the above table has the word 'Y' (and) or 'O' (or). This words should not be considered in our future model.

To do this we will use the Natural Language Toolkit (NLTK).

First, let's define a function to normalize strings. Here, we don't care about accents*, punctuations and non-alphabetic characters in general, and lower/upper case, so we will get rid of that
*while it's true that in Spanish an accent can change the meaning of a word, this doesn't usually happen with nouns, and we have to take into account that is probable that most words that have an accent are probably written with and without it inside the dataset

In [24]:
import unidecode
import re


def to_alpha_lower_ascii(val: str) -> str:
    ascii_value = unidecode.unidecode(val)
    lower = ascii_value.lower()
    alpha = re.sub(r'[^a-z]', ' ', lower)
    alpha_spaces = re.sub(r'\s+', ' ', alpha)
    return alpha_spaces

In [25]:
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords

sw_nltk = [to_alpha_lower_ascii(x) for x in stopwords.words('spanish')]
print("The words considered stopwords in spanish are: ")
print(sw_nltk)

The words considered stopwords in spanish are: 
['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'mas', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'si', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'tambien', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mi', 'antes', 'algunos', 'que', 'unos', 'yo', 'otro', 'otras', 'otra', 'el', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tu', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mio', 'mia', 'mios', 'mias', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\spost\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Now let's convert the values to simple ascii using the previous function and get rid of stopwords and other short words (1 or 2 letters)

In [26]:
def clean_str(value: str) -> str:
    converted_words = to_alpha_lower_ascii(value).split()
    return ' '.join(word for word in converted_words if word not in sw_nltk and len(word) > 2)

In [27]:
data['Descripcion de la Partida Aduanera_clean'] = data['Descripcion de la Partida Aduanera'].apply(clean_str)
data[['Descripcion de la Partida Aduanera', 'Descripcion de la Partida Aduanera_clean']]

,Descripcion de la Partida Aduanera,Descripcion de la Partida Aduanera_clean
0,"ACEITUNA CONSERVADAS PROVISIONALMENTE,TODAVIA ...",aceituna conservadas provisionalmente todavia ...
1,"ACEITUNA CONSERVADAS PROVISIONALMENTE,TODAVIA ...",aceituna conservadas provisionalmente todavia ...
2,"ACEITUNA CONSERVADAS PROVISIONALMENTE,TODAVIA ...",aceituna conservadas provisionalmente todavia ...
3,ACEITUNAS,aceitunas
4,"ACEITUNA CONSERVADAS PROVISIONALMENTE,TODAVIA ...",aceituna conservadas provisionalmente todavia ...
...,...,...
631389,CALABAZAS (ZAPALLOS) Y CALABACINES (CUCURBITA ...,calabazas zapallos calabacines cucurbita spp
631390,CALABAZAS (ZAPALLOS) Y CALABACINES (CUCURBITA ...,calabazas zapallos calabacines cucurbita spp
631391,CALABAZAS (ZAPALLOS) Y CALABACINES (CUCURBITA ...,calabazas zapallos calabacines cucurbita spp
631392,CALABAZAS (ZAPALLOS) Y CALABACINES (CUCURBITA ...,calabazas zapallos calabacines cucurbita spp


In [28]:
data.drop('Descripcion de la Partida Aduanera', axis=1, inplace=True)
data.rename({'Descripcion de la Partida Aduanera_clean': 'Descripcion de la Partida Aduanera'}, axis=1, inplace=True)

#### **Fecha (date)**

For this column we will map the month of shipment

In [29]:
data['Fecha'] = pd.to_datetime(data['Fecha'].values, infer_datetime_format=True).month

#### **Kg Neto (net weight in of good KG) and U$ FOB Tot (total price of good)**

As we said before, here we will get the price by kg of the good. To do this we will use both columns and transform them into one.

In [30]:
import numpy as np

data['Kg Neto'] = data['Kg Neto'].str.replace(',', '.').astype(float).values
data['U$ FOB Tot'] = data['U$ FOB Tot'].str.replace(',', '.').astype(float).values

In [31]:
(data["Kg Neto"] == 0).value_counts()

False    631332
True         62
Name: Kg Neto, dtype: int64

In [32]:
data = data.drop(data[data["Kg Neto"] == 0].index)

In [33]:
data["usd_kg"] = np.divide(data['Kg Neto'], data['U$ FOB Tot'])
data['usd_kg'] = data['usd_kg'].fillna(0)
data['usd_kg'] = data['usd_kg'].replace([[np.inf, -np.inf]], 0)

data = data.drop(columns=["Kg Neto", "U$ FOB Tot"])

In [34]:
data['usd_kg'].describe()

count    631332.000000
mean          5.460213
std         271.044459
min           0.000000
25%           0.222281
50%           0.371321
75%           0.585714
max       24000.000000
Name: usd_kg, dtype: float64

#### **País de destino (country of destiny)**

In [35]:
countries = data["Pais de Destino"].unique()
countries.sort()
countries

array(['Aguas Internacionales', 'Albania', 'Alemania', 'Arabia Saudita',
       'Argelia', 'Argentina', 'Aruba', 'Australia', 'Austria', 'Bahamas',
       'Bahréin', 'Barbados', 'Benín', 'Bielorrusia', 'Bolivia',
       'Bosnia Y Herzegovina', 'Brasil', 'Bulgaria', 'Bélgica',
       'Cabo Verde', 'Camboya', 'Camerún', 'Canadá', 'Chile', 'China',
       'Chipre', 'Colombia', 'Corea Del Norte', 'Corea Del Sur',
       'Costa De Marfil', 'Costa Rica', 'Croacia', 'Cuba', 'Curazao',
       'Dinamarca', 'Ecuador', 'Egipto', 'El Salvador',
       'Emiratos Árabes Unidos', 'Eslovenia', 'España', 'Estados Unidos',
       'Estonia', 'Federación Rusa', 'Filipinas', 'Finlandia', 'Francia',
       'Gabón', 'Georgia', 'Ghana', 'Grecia', 'Groenlandia', 'Guadalupe',
       'Guatemala', 'Guayana Francesa', 'Haití', 'Honduras', 'Hong Kong',
       'Hungría', 'India', 'Indonesia', 'Irak', 'Irlanda',
       'Irán (República Islámica De)',
       'Islas Ultramarinas Menores De Estados Unidos', 'Israel', 'I

The column is correct and shows not corrupted data. We will only set the values to lowercase and remove accents.

In [36]:
data["Pais de Destino"] = data["Pais de Destino"].apply(lambda country: unidecode.unidecode(country).lower())

#### **Descripcion Comercial (comercial description)**

As it will be shown below, there are values in these columns with repeated sentences inside

In [37]:
comercial_description = data["Descripcion Comercial"].tolist()
comercial_description[0]

'ACEITUNAS ARAUCO PRESENTACION EN BIDONES   RESTITUCION DERECHOS ARANC. (COD 13)'

Let's clean this and also remove accents, double or more white spaces, stopwords, punctuations and set to lowercase

In [38]:
def remove_repetitions(source: str) -> str:
    return re.match(r'^\s*([\w\s]+?)(?:\s*\1)*\s*$', source)[1]

In [39]:
data['Descripcion Comercial'] = data['Descripcion Comercial'].apply(clean_str).apply(remove_repetitions)
data['Descripcion Comercial']

0         aceitunas arauco presentacion bidones restituc...
1         aceitunas arauco presentacion bidones verde sa...
2         bidones aceituna mesa ascolana verde deshuesad...
3         aceituna ascolana entera bid acoge restitucion...
4         aceituna ascolano presentacion bidones restitu...
                                ...                        
631389                              zapallo calabaza fresco
631390                                              zapallo
631391      zapallocodigo restitucion derechos arancelarios
631392    zapallos pumpkins zapallos frescos acta inspec...
631393                         zapallos congelados big bags
Name: Descripcion Comercial, Length: 631332, dtype: object

#### **Categoria macro Aurum (subcategories)**

This is the column to predict

Finally, our data is clean and ready to be preprocessed. As a last step, we will reset the indexes.

In [40]:
data.reset_index(drop=True, inplace=True)
data = data[[col for col in data if col != 'Categoría macro Aurum'] + ['Categoría macro Aurum']]
data.to_csv('../data/cleaned_data/cleaned_data.csv', index=False)
data

,Fecha,Pais de Destino,Descripcion Comercial,Descripcion de la Partida Aduanera,usd_kg,Categoría macro Aurum
0,11,chile,aceitunas arauco presentacion bidones restituc...,aceituna conservadas provisionalmente todavia ...,0.703116,Aceituna
1,2,chile,aceitunas arauco presentacion bidones verde sa...,aceituna conservadas provisionalmente todavia ...,0.844524,Aceituna
2,10,australia,bidones aceituna mesa ascolana verde deshuesad...,aceituna conservadas provisionalmente todavia ...,0.277778,Aceituna
3,7,jordania,aceituna ascolana entera bid acoge restitucion...,aceitunas,0.434783,Aceituna
4,11,chile,aceituna ascolano presentacion bidones restitu...,aceituna conservadas provisionalmente todavia ...,0.760793,Aceituna
...,...,...,...,...,...,...
631327,5,espana,zapallo calabaza fresco,calabazas zapallos calabacines cucurbita spp,0.714286,Zapallo
631328,7,espana,zapallo,calabazas zapallos calabacines cucurbita spp,0.666667,Zapallo
631329,5,suiza,zapallocodigo restitucion derechos arancelarios,calabazas zapallos calabacines cucurbita spp,0.416667,Zapallo
631330,4,alemania,zapallos pumpkins zapallos frescos acta inspec...,calabazas zapallos calabacines cucurbita spp,1.000000,Zapallo
